# ہوٹل بکنگ کے لیے ترجیحی ممبر مڈل ویئر

یہ نوٹ بک **فنکشن پر مبنی مڈل ویئر** کو مائیکروسافٹ ایجنٹ فریم ورک کے ذریعے پیش کرتی ہے۔ ہم مشروط ورک فلو کی مثال کو ایک مڈل ویئر لیئر شامل کرکے بڑھاتے ہیں جو ترجیحی ممبرز کو خاص مراعات فراہم کرتی ہے۔

## آپ کیا سیکھیں گے:
1. **فنکشن پر مبنی مڈل ویئر**: فنکشن کے نتائج کو روکنا اور ان میں ترمیم کرنا
2. **کانٹیکسٹ تک رسائی**: `context.result` کو عمل درآمد کے بعد پڑھنا اور تبدیل کرنا
3. **کاروباری منطق کا نفاذ**: ترجیحی ممبرز کے فوائد
4. **نتائج کی اووررائیڈ**: صارف کی حیثیت کی بنیاد پر فنکشن کے نتائج کو تبدیل کرنا
5. **ایک ہی ورک فلو، مختلف نتائج**: مڈل ویئر کے ذریعے رویے میں تبدیلی

## مڈل ویئر کے ساتھ ورک فلو کی ساخت:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## مشروط ورک فلو سے اہم فرق:

**مڈل ویئر کے بغیر** (14-conditional-workflow.ipynb):
- پیرس میں کمرے دستیاب نہیں → alternative_agent کی طرف بھیجنا

**مڈل ویئر کے ساتھ** (یہ نوٹ بک):
- عام صارف + پیرس → کمرے دستیاب نہیں → alternative_agent کی طرف بھیجنا
- ترجیحی صارف + پیرس → 🌟 مڈل ویئر اووررائیڈ کرتا ہے! → دستیاب → booking_agent کی طرف بھیجنا

## ضروریات:
- مائیکروسافٹ ایجنٹ فریم ورک انسٹال ہو
- مشروط ورک فلو کی سمجھ (دیکھیں 14-conditional-workflow.ipynb)
- GitHub ٹوکن یا OpenAI API کی
- مڈل ویئر پیٹرنز کی بنیادی سمجھ


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## مرحلہ 1: ساختی نتائج کے لیے Pydantic ماڈلز کی تعریف کریں

یہ ماڈلز وہ **اسکیمہ** متعین کرتے ہیں جو ایجنٹس واپس کریں گے۔ ہم نے `priority_override` فیلڈ شامل کی ہے تاکہ معلوم ہو سکے کہ کب مڈل ویئر دستیابی کے نتیجے کو تبدیل کرتا ہے۔


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## مرحلہ 2: ترجیحی ممبرز ڈیٹا بیس کی وضاحت کریں

اس ڈیمو کے لیے، ہم ترجیحی ممبرشپ ڈیٹا بیس کی نقل کریں گے۔ پروڈکشن میں، یہ ایک حقیقی ڈیٹا بیس یا API سے کوئری کرے گا۔

**ترجیحی ممبرز:**
- `alice@example.com` - وی آئی پی ممبر
- `bob@example.com` - پریمیم ممبر  
- `priority_user` - ٹیسٹ اکاؤنٹ


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## مرحلہ 3: ہوٹل بکنگ ٹول بنائیں

بالکل ویسے ہی جیسے مشروط ورک فلو، لیکن اب یہ مڈل ویئر کے ذریعے روکا جائے گا!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## مرحلہ 4: 🌟 ترجیحی چیک مڈل ویئر بنائیں (اہم خصوصیت!)

یہ اس نوٹ بک کی **بنیادی خصوصیت** ہے۔ مڈل ویئر:

1. **روک لیتا ہے** hotel_booking فنکشن کال کو
2. **عام طور پر انجام دیتا ہے** فنکشن کو `next(context)` کال کرکے
3. **نتیجہ کا جائزہ لیتا ہے** `context.result` میں
4. **نتیجہ کو تبدیل کرتا ہے** اگر صارف ترجیحی ہو اور کمرے دستیاب نہ ہوں
5. **تبدیل شدہ نتیجہ واپس کرتا ہے** ایجنٹ کو

**اہم پیٹرن:**
```python
async def my_middleware(context, next):
    await next(context)  # Execute function
    # Now context.result contains the function's output
    if some_condition:
        context.result = new_value  # Override!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## مرحلہ 5: روٹنگ کے لیے شرطی فنکشنز کی تعریف کریں

وہی شرطی فنکشنز جو مشروط ورک فلو میں استعمال ہوتے ہیں - یہ منظم آؤٹ پٹ کا جائزہ لیتے ہیں تاکہ روٹنگ کا تعین کیا جا سکے۔


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## مرحلہ 6: کسٹم ڈسپلے ایگزیکیوٹر بنائیں

پہلے کی طرح کا ہی ایگزیکیوٹر - حتمی ورک فلو آؤٹ پٹ دکھاتا ہے۔


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## مرحلہ 7: ماحول کے متغیرات لوڈ کریں

LLM کلائنٹ (GitHub Models یا OpenAI) کو ترتیب دیں۔


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## مرحلہ 8: AI ایجنٹس کو مڈل ویئر کے ساتھ بنائیں

**اہم فرق:** جب availability_agent بنایا جاتا ہے، تو ہم `middleware` پیرامیٹر پاس کرتے ہیں!

یہی طریقہ ہے جس سے ہم priority_check_middleware کو ایجنٹ کے فنکشن کال کے عمل میں شامل کرتے ہیں۔


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## مرحلہ 9: ورک فلو بنائیں

پہلے کی طرح ہی ورک فلو کا ڈھانچہ - دستیابی کی بنیاد پر مشروط راستہ۔


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## مرحلہ 10: ٹیسٹ کیس 1 - پیرس میں عام صارف (کوئی اووررائیڈ نہیں)

ایک عام صارف پیرس میں بکنگ کی کوشش کرتا ہے → کوئی کمرے دستیاب نہیں → متبادل ایجنٹ کی طرف بھیج دیا جاتا ہے


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## مرحلہ 11: ٹیسٹ کیس 2 - 🌟 پیرس میں ترجیحی صارف (اووررائیڈ کے ساتھ!)

ایک ترجیحی ممبر پیرس میں بکنگ کی کوشش کرتا ہے → ابتدائی طور پر کوئی کمرے دستیاب نہیں → 🌟 مڈل ویئر اووررائیڈ کرتا ہے! → بکنگ ایجنٹ کی طرف بھیجتا ہے

**یہ مڈل ویئر کی طاقت کا اہم مظاہرہ ہے!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## مرحلہ 12: ٹیسٹ کیس 3 - اسٹاک ہوم میں ترجیحی صارف (پہلے سے دستیاب)

ترجیحی صارف اسٹاک ہوم کی کوشش کرتا ہے → کمرے دستیاب ہیں → اووررائیڈ کی ضرورت نہیں → booking_agent کی طرف بھیج دیا جاتا ہے

یہ ظاہر کرتا ہے کہ مڈل ویئر صرف ضرورت پڑنے پر ہی کام کرتا ہے!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## اہم نکات اور مڈل ویئر کے تصورات

### ✅ آپ نے کیا سیکھا:

#### **1. فنکشن پر مبنی مڈل ویئر پیٹرن**

مڈل ویئر فنکشن کالز کو ایک سادہ async فنکشن کے ذریعے روکتا ہے:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Before function execution
    print("Intercepting...")
    
    # Execute the function
    await next(context)
    
    # After function execution - inspect result
    if context.result:
        # Modify result if needed
        context.result = modified_value
```

#### **2. کانٹیکسٹ تک رسائی اور نتیجہ کو اووررائیڈ کرنا**

- `context.function` - کال کیے جانے والے فنکشن تک رسائی حاصل کریں
- `context.arguments` - فنکشن کے دلائل پڑھیں
- `context.kwargs` - اضافی پیرامیٹرز تک رسائی حاصل کریں
- `await next(context)` - فنکشن کو چلائیں
- `context.result` - فنکشن کے آؤٹ پٹ کو پڑھیں/ترمیم کریں

#### **3. بزنس لاجک کا نفاذ**

ہمارا مڈل ویئر ترجیحی ممبر فوائد کو نافذ کرتا ہے:
- **عام صارفین**: کوئی ترمیم نہیں، معیاری ورک فلو
- **ترجیحی صارفین**: "کوئی دستیابی نہیں" کو اووررائیڈ کریں → "دستیاب"
- **مشروط منطق**: صرف ضرورت پڑنے پر اووررائیڈ کریں

#### **4. ایک ہی ورک فلو، مختلف نتائج**

مڈل ویئر کی طاقت:
- ✅ ورک فلو کے ڈھانچے میں کوئی تبدیلی نہیں
- ✅ ٹول فنکشن میں کوئی تبدیلی نہیں
- ✅ مشروط روٹنگ منطق میں کوئی تبدیلی نہیں
- ✅ صرف مڈل ویئر → مختلف رویہ!

### 🚀 حقیقی دنیا کی ایپلیکیشنز:

1. **وی آئی پی/پریمیم خصوصیات**
   - پریمیم صارفین کے لیے ریٹ کی حد کو اووررائیڈ کریں
   - وسائل تک ترجیحی رسائی فراہم کریں
   - پریمیم خصوصیات کو متحرک طور پر ان لاک کریں

2. **A/B ٹیسٹنگ**
   - صارفین کو مختلف نفاذات پر بھیجیں
   - مخصوص صارفین کے ساتھ نئی خصوصیات کی جانچ کریں
   - خصوصیات کو بتدریج رول آؤٹ کریں

3. **سیکیورٹی اور تعمیل**
   - فنکشن کالز کا آڈٹ کریں
   - حساس آپریشنز کو بلاک کریں
   - بزنس رولز نافذ کریں

4. **کارکردگی کی اصلاح**
   - مخصوص صارفین کے لیے نتائج کو کیش کریں
   - مہنگے آپریشنز کو ممکنہ حد تک چھوڑ دیں
   - وسائل کی متحرک تقسیم

5. **غلطی سے نمٹنا اور دوبارہ کوشش کرنا**
   - غلطیوں کو خوش اسلوبی سے پکڑیں اور سنبھالیں
   - دوبارہ کوشش کرنے کی منطق نافذ کریں
   - متبادل نفاذات پر واپس جائیں

6. **لاگنگ اور مانیٹرنگ**
   - فنکشن کے عمل درآمد کے اوقات کو ٹریک کریں
   - پیرامیٹرز اور نتائج کو لاگ کریں
   - استعمال کے نمونوں کی نگرانی کریں

### 🔑 ڈیکوریٹرز سے اہم فرق:

| خصوصیت | ڈیکوریٹر | مڈل ویئر |
|---------|-----------|------------|
| **دائرہ کار** | ایک فنکشن | ایجنٹ کے تمام فنکشنز |
| **لچک** | تعریف پر مقرر | رن ٹائم پر متحرک |
| **کانٹیکسٹ** | محدود | مکمل ایجنٹ کانٹیکسٹ |
| **کمپوزیشن** | متعدد ڈیکوریٹرز | مڈل ویئر پائپ لائن |
| **ایجنٹ سے آگاہ** | نہیں | ہاں (ایجنٹ کی حالت تک رسائی) |

### 📚 مڈل ویئر کب استعمال کریں:

✅ **مڈل ویئر استعمال کریں جب:**
- آپ کو صارف/سیشن کی حالت کی بنیاد پر رویہ تبدیل کرنے کی ضرورت ہو
- آپ کو متعدد فنکشنز پر منطق لاگو کرنی ہو
- آپ کو ایجنٹ لیول کانٹیکسٹ تک رسائی کی ضرورت ہو
- آپ کراس کٹنگ خدشات (لاگنگ، تصدیق وغیرہ) نافذ کر رہے ہوں

❌ **مڈل ویئر استعمال نہ کریں جب:**
- سادہ ان پٹ کی توثیق (Pydantic استعمال کریں)
- فنکشن مخصوص منطق (فنکشن میں رکھیں)
- ایک بار کی ترمیمات (بس فنکشن کو تبدیل کریں)

### 🎓 جدید پیٹرنز:

```python
# Multiple middleware (execution order matters!)
middleware=[
    logging_middleware,      # Logs first
    auth_middleware,         # Then checks auth
    cache_middleware,        # Then checks cache
    rate_limit_middleware,   # Then rate limits
    priority_check_middleware  # Finally priority check
]

# Conditional middleware execution
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modify result
    else:
        # Skip execution entirely
        context.result = cached_value
```

### 🔗 متعلقہ تصورات:

- **ایجنٹ مڈل ویئر**: ایجنٹ.run() کالز کو روکتا ہے
- **فنکشن مڈل ویئر**: ٹول فنکشن کالز کو روکتا ہے (جو ہم نے استعمال کیا!)
- **مڈل ویئر پائپ لائن**: ترتیب میں مڈل ویئر کا سلسلہ
- **کانٹیکسٹ پروپیگیشن**: مڈل ویئر چین کے ذریعے حالت منتقل کریں



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
